Tools

In [1]:
%pip install seaborn

   ---------------------------------------- 0.0/294.6 kB ? eta -:--:--
   --------------------------------------- 294.6/294.6 kB 17.8 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style("darkgrid")